![Practicum AI Logo image](https://github.com/PracticumAI/practicumai.github.io/blob/main/images/logo/PracticumAI_logo_250x50.png?raw=true) <img src='https://github.com/PracticumAI/practicumai.github.io/blob/main/images/icons/practicumai_beginner.png?raw=true' align='right' width=50>
***
# *Practicum AI:* Deep Learning Basics

This exercise is inspired by Baig et al. (2020) <i>The Deep Learning Workshop</i> from <a href="https://www.packtpub.com/product/the-deep-learning-workshop/9781839219856">Packt Publishers</a> (Exercise 1.01, page 7).

## Deep learning for image recognition

Before diving into exactly _how_ deep learning works, let's explore it through an example. We will explore just a few of the fantastic things you can do with existing models and see how easy it can be to implement AI tools with code. 

 In this exercise, we will use a pre-trained deep learning model, [ResNet50](https://arxiv.org/abs/1512.03385), which has been trained on [ImageNet](https://image-net.org/), a collection of about 1.3 million images labeled as being in one of 1,000 categories. We won't focus on the details of ResNet50, but understand that it is a deep learning image classification model that was developed to categorize images into one of the 1,000 categories found in the ImageNet dataset.

To help with this exercise, let us introduce you to Amelia. <img alt='A GAN-generated cartoon image of Amelia, our character for this exercise.' src='images/amelia.png' align='right' width=300 style="float: right; padding: 10px 0px 0px 10px">

Amelia, our teenaged, tech geek heroine, is an extremely picky eater. She's something of a pizza fanatic 🍕! When dinner time rolls around, she only wants to come down from her bedroom if pizza is on the menu. 

Being a tech geek, Amelia has installed a hidden camera in the dining room.  Once dinner is set, the camera snaps a photo of the meal.  Now, she wants to develop an AI image recognition system to determine whether pizza is served.  

Let's help Amelia code her pizza recognition system!


### <img src='images/note_icon.svg' width=40, align='center' alt='Note icon'> Note

> As a side note, the drawing of Amelia was generated by a Generative Adversarial Network (GAN) trained on Anime images and created at: [https://thisanimedoesnotexist.ai/](https://thisanimedoesnotexist.ai/). To learn more about GANs, check out our [Practicum AI GAN learning experience](https://github.com/PracticumAI/gan).

Amelia is a *Practicum AI* alumnus and recalls the AI Application Development Pathway. She will use that in creating her pizza recognition system.

![Practicum AI Application Pathway Image](images/application_dev_pathway.png) 

With her pizza-recognizing hidden camera, she's already completed Step 1: Choose a Problem! Due to the flexible nature of coding, implementing the next six steps will jump around a bit. Don't worry though, Amelia knows her stuff and will ensure we know where we're at in the application process. Here's the overview of the steps in the application development process and how they correspond to the code in this Jupyter Notebook:
1. Choose a Problem - Making a pizza recognizer!
2. Gather Good Data - Amelia is just a teenager; she can't afford to take thousands of images of pizza and other objects and train a model from scratch! Instead, she will use a model that's already been trained to "recognize" pizza (and 999 additional categories).
3. Clean and Prep Data - Amelia's selected model was already trained on the large set of images in ImageNet, so she doesn't need to worry about prepping her training data. She will, however, need to do some work to ensure that her new inputs are formatted correctly.
4. Choose a Model - Amelia realizes that she can take advantage of a model that's already trained, and doesn't need to start from scratch.

### <img src='images/note_icon.svg' width=40, align='center' alt='Note icon'> Note

> While you may not be looking to classify pizza images (or have a very different AI project in mind) note that in most cases, there are probably models you can build from. The relative openness of AI researchers in sharing their models has enabled the community to:
>   * Use a trained model "out of the box" as Amelia will.
>   * Fine tune a model with some of your data.
>   * Use an existing model architecture and train with your data.
>   * Modify an existing model architecture and train with your data.
> Rarely is there a need to start from scratch!

5. Train the Model - Our heroine is going to use a model that's pre-trained so... Done! She's up and running with an AI application without needing to compile or train anything. Magic!
6. Evaluate the Model - As part of the evaluation process, Amelia will need to test the model to see how well it recognizes pizza.
7. Deploy the Model - Since Amelia is comfortable using Jupyter Notebooks, she's going to leave the application here. Embedding the model in another application is unnecessary (and beyond the scope of this course!)

## 1. Import libraries

Import the necessary libraries. For this exercise, Amelia will use the pre-trained ResNet50 model that is part of Keras: `from tensorflow.keras.applications.resnet50 import ResNet50`. Check out the [Keras documentation](https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet50/ResNet50) for more details. 

In [1]:
# Import necessary libraries for image processing and deep learning. The image processing functions, like img_to_array, will help Amelia format the image to run through her model.

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import decode_predictions

# Import base tensorflow and set seed to achieve consistent results.
import tensorflow as tf 
import numpy as np

seed = 42  # Set the seed for reproducibility

tf.random.set_seed(seed)
np.random.seed(seed)


#### 2. Instantiate the Resnet50 model

Instantiating is a programming term that means taking the 'blueprint' of something (in this case, ResNet50) and making an object out of it (the model we will use here). This step creates the instance of the model to use.

### <img src='images/note_icon.svg' width=40, align='center' alt='Note icon'> Some Background on ResNet 
> ResNet, which stands for Residual Network, won the 2015 ImageNet competition. It was introduced to address the vanishing gradient problem commonly faced when training very deep neural networks. As networks become deeper, gradients (the values used to update network weights) can become extremely small, effectively halting training. 
> ResNet introduces the concept of "residual blocks."  As it processes data, instead of relying solely on the current "thought" or layer, it can also "refer back" to earlier layers, much like using recent memories to help recall older ones. These "references back" are called skip connections. They act like bridges, letting the network jump over some layers to ensure that even as it delves deeper into processing, it doesn't forget or lose important early details. This shortcut or skip connection allows gradients to propagate more easily through the network. 
> This architectural innovation has enabled the training of networks with depths previously thought unfeasible. ResNet models, with hundreds or even thousands of layers, have achieved state-of-the-art performance on many image classification benchmarks. 
> In this unit’s exercise we used the ResNet50 model, which as its name suggests, consists of 50 layers.

<div style="padding: 10px; margin-bottom: 20px; border: thin solid #E5C250; border-left-width: 10px;background-color: #fff"><strong>Tip:</strong> You will likely see some output highlighted in red. While red is used for errors, it is also used for warnings. It can take some getting used to, but red is OK in this case...</div>

```python
mymodel = ResNet50() # Create an instance of the ResNet50 model pre-trained on ImageNet data
```

In [2]:
# Code it!


#### 3. Load image

Amelia has a test image of her favorite pizza to use to test the system. Let's load her pizza image in.

Since ResNet50 was trained using images that are 224X224 pixels, we need to transform the input image to be the same size.

<div style="padding: 10px; margin-bottom: 20px; border: thin solid #E5C250; border-left-width: 10px;background-color: #fff"><strong>Tip:</strong> The pizza image is stored in the images folder, the complete path of the location where the image is located must be given.
<br><br>
If running on Google Colab, you have a couple of choices:

1. Change the `'images/pizza.jpg'` to use the web location of the image in this repository: `'https://raw.githubusercontent.com/PracticumAI/deep_learning/main/images/pizza.jpg'`
1. Find any image of pizza, and upload it using the Files tab as in the image below. Then right click and "Copy path", and use that path.
<br>
<img src='images/colab_img_upload.png' alt='Screenshot of image upload for Google Colab'>
    
</div>

```python
myimage = load_img('images/pizza.jpg', target_size = (224, 224)) # Load an image file for testing, resizing it to the required input size of 224x224 pixels
```


In [3]:
# Code it!


#### 4. View the pizza image

Let's take a quick look at the image to verify that it's a pizza.  Type the variable name and run the code block.

```python
myimage
```

In [1]:
# Code it!



#### 5. Convert image to array

Convert the image to an array because the model expects it in this format.

```python
myimage = img_to_array(myimage) # Convert the loaded image to an array format suitable for processing
```

In [5]:
# Code it!


#### 6. Reshape image

Reshape the image.  All images fed to this model need to be 224 pixels high and 224 pixels wide, with 3 channels, one for each color (Red, Green, Blue).  If our image was greyscale, how many channels would we specify?

```python
myimage = myimage.reshape((1, 224, 224, 3)) # Reshape the image array to the format the model expects (batch size, height, width, color channels)
```

In [6]:
# Code it!


#### 7. Pre-process image

Execute the *preprocess_image()* function with the image.

```python
myimage = preprocess_input(myimage) # Preprocess the image to ensure its values are appropriate for the ResNet50 model
```

In [7]:
# Code it!


#### 8. Execute predict method

Execute the model's predict method.

```python
myresult = mymodel.predict(myimage) # Use the model to predict the class (or category) of the image
```

In [2]:
# Code it!


#### 9. Get prediction label

The model's predict method returns a number.  Convert this to its corresponding text label.

```python
mylabel = decode_predictions(myresult) # Decode the prediction result to get human-readable class labels
```

In [25]:
# Code it!


#### 10. Assign list item to variable 

Assign the first item listed by the prediction to a variable - this is the label with the highest probability.

```python
# Extract the label with the highest predicted probability. 
# Recalling that in Python, all indexes start at 0, he [0][0] indexing retrieves the first prediction from the first batch of results.
mylabel = mylabel[0][0] 
```

In [26]:
# Code it!


#### 11. Embed label 

Embed the label in a sentence and then print it.

```python
# The 'mylabel' variable contains information about the prediction in the format (ID, Label, Probability).
# Using 'mylabel[1]' extracts the human-readable label (e.g., 'pizza') for the predicted class.
print("This is an image of a " + mylabel[1]) # Print the predicted class label in a formatted string

```

In [3]:
# Code it!


<div style="padding: 10px;margin-bottom: 20px;border: thin solid #E5C250;border-left-width: 10px;background-color: #fff"><strong>Tip:</strong> Although we use an image of a pizza here, you can use just about any image with this model. Try out this exercise multiple times with different images to see if you can fool it. The <a href='https://raw.githubusercontent.com/PracticumAI/deep_learning/main/resnet_labels.txt'>resnet_labels.txt</a> file lists all the images this model is trained to classify.</div>

#### 12. Create a speech sentence

Create a longer sentence to convert to speech. Amelia wants her model to output an audio file to tell her the results so she doesn't even have to leave her beanbag chair to know what's for dinner.

```python
sayit = "This is an image of a " + mylabel[1] + " in full living color."
```

In [28]:
# Code it!


#### 13. Import gtts libraries

Import the required libraries.  Google Text to Speech (gtts) is an open source cloud-based application programming interface (API) that... Converts text to speech.

In [30]:
from gtts import gTTS
import os

# If you're using Google Colab, sometimes gTTS won't import. If you get an import error, use the following code (without the hashtag, of course) to install gTTS:
# !pip install gTTS

#### 14. Execute the gtts function

Pass the sayit variable to the gTTS API.

```python
myobj = gTTS(text = sayit)
```

In [32]:
# Code it!


#### 15. Save the audio file

gTTS will convert the string you gave it into an audio file. Save the audio file. The default location is the current directory.

```python
myobj.save("prediction.mp3") # Save the audio file in the current directory.
```

In [33]:
# Code it!


<div style="padding: 10px;margin-bottom: 20px;border:  thin solid #30335D; border-left-width: 10px;background-color: #fff"><strong>Note:</strong> This last block of code is only needed if you are running Jupyter Notebooks on a local computer.  Otherwise, download the .mp3 file from HiperGator and listen to it on your computer.</div>

In [ ]:
# Uncomment and run if running on local system as opposed to HPG or Google Colab
# os.system("prediction.mp3")

#### 16. Let's put it all together

We can put all of these steps together in a function to make it easier to test more images.

In [38]:
# Define a function that automates the process of loading, processing, and predicting the class of an image
def whats_for_dinner(image): 
    myimage = load_img(image, target_size = (224, 224))
    myimage = img_to_array(myimage)
    myimage = myimage.reshape((1, 224, 224, 3))
    myimage = preprocess_input(myimage)
    myresult = mymodel.predict(myimage)
    mylabel = decode_predictions(myresult)
    toplabel = mylabel[0][0]

    if toplabel[1] == 'pizza':
        sayit = "Amelia, tonight's dinner is " + toplabel[1] + ", let's go downstairs for dinner!"
    else:
        sayit = "Amelia, tonight's dinner is " + toplabel[1] + ", let's stay here and order a pizza......!"

    myobj = gTTS(text = sayit)

    return mylabel, myobj

In [39]:
label, soundclip2 = whats_for_dinner('images/220px-Ham_(4).jpg')
print(label)

soundclip2.save("prediction2.mp3")

1/1 [==============================] - 0s 64ms/step
[[('n02281787', 'lycaenid', 0.26136062), ('n03804744', 'nail', 0.11522047), ('n02277742', 'ringlet', 0.055332493), ('n07836838', 'chocolate_sauce', 0.051150553), ('n07684084', 'French_loaf', 0.034306075)]]


Amelia should test with non-pizza meals too to make sure her system is working. Let's try this burger. ![Photo of a hamburger](images/hamburger.jpg)

In [ ]:
label, soundclip3 = whats_for_dinner('images/hamburger.jpg')
print(label)

soundclip3.save("prediction3.mp3")

It looks like Amelia's classifier is working well as it predicted that the image was a cheeseburger. 

But Amelia's parents are on to her...they started making food that looks like pizza, but isn't! How about this quiche? Does it fool the AI? Can you find images that trick Amelia into coming down for dinner?

![A photo of quiche](images/quiche.jpg)

In [ ]:
label, soundclip4 = whats_for_dinner('images/quiche.jpg')
print(label)

soundclip4.save("prediction4.mp3")